In [57]:
import torch
import numpy as np
from torch.utils.data import Dataset,DataLoader
from torch import nn
import dask.array as da

In [59]:
dataList = [da.from_npy_stack(f"D:/RnnMachineLearning/stack2/sub-{str(sub).zfill(3)}/") for sub in range(1,6)]
# data = np.load('D:\RnnMachineLearning\meditation_data.npy',allow_pickle=True)

In [68]:
data = da.concatenate(dataList, axis=0)
a =data[0]
a.compute_chunk_sizes()

ValueError: Array can't be memory-mapped: Python objects in dtype.

In [6]:
filterArrtrain = []
filterArrtest = []
for i in data:
    if i[0] in ("001","002","003"):
        filterArrtrain.append(True)
        filterArrtest.append(False)
    else:
        filterArrtest.append(True)
        filterArrtrain.append(False)

arrtrain = data[filterArrtrain]
arrtest = data[filterArrtest]

In [8]:
class EegDataSet(Dataset):

	def __init__(self,dataArray):
		#data loading
		self.x = torch.from_numpy(np.stack(dataArray[:,1]))
		self.labels = torch.from_numpy(dataArray[:,2].astype("float32"))

	def __len__(self):
		# len(dataset)
		return len(self.labels)

	def __getitem__(self, idx):
		return self.x[idx], self.labels[idx]

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# HYPERPARAMETERS
hidden_size = 128
num_classes = 2
num_epochs = 1
batch_size = 20
learning_rate = 0.001

input_size = 72
sequence_length = 10240
num_layers = 2

dataArrTrain = EegDataSet(arrtrain)
dataArrTest = EegDataSet(arrtest)

dlDataArrTrain = DataLoader(dataArrTrain, batch_size=batch_size, shuffle=True)
dlDataArrtest = DataLoader(dataArrTest, batch_size=batch_size, shuffle=True)

In [33]:
# NETWORK

# Vanilla RNN
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first = True)
        # x -> (batch_size, sequence_length, input_size)

        # or:
        #self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        #self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)

        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # print(x.shape)
        x = torch.reshape(x,(x.shape[0],x.shape[2],x.shape[1]))
        # print(x.shape)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        # print(x.shape)
        # print(h0.shape)
        out, _ = self.rnn(x, h0)
        # out -> (batch_size, sequence_length, hidden_size)
        # out -> (N, 129, 128) ->> NOTE: CHANGE SEQUENCE_LENGTH AS 750 LATER (TRANPOSE THE TENSOR)
        out = out[:, -1, :]
        # out -> (N, 128)
        out = self.fc(out)
        return out


In [34]:
model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

In [35]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [36]:
# Train the model
n_total_steps = len(dlDataArrTrain)
for epoch in range(num_epochs):
    for i, data_b in enumerate(dlDataArrTrain):

        inputs, labels = data_b[0].to(torch.float32),data_b[1].to(torch.int64)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

Epoch [1/1], Step [1/37], Loss: 0.6758
Epoch [1/1], Step [2/37], Loss: 0.6909
Epoch [1/1], Step [3/37], Loss: 0.8045
Epoch [1/1], Step [4/37], Loss: 0.7722
Epoch [1/1], Step [5/37], Loss: 0.7057
Epoch [1/1], Step [6/37], Loss: 0.6930
Epoch [1/1], Step [7/37], Loss: 0.6907
Epoch [1/1], Step [8/37], Loss: 0.6995
Epoch [1/1], Step [9/37], Loss: 0.6864
Epoch [1/1], Step [10/37], Loss: 0.6921
Epoch [1/1], Step [11/37], Loss: 0.6836
Epoch [1/1], Step [12/37], Loss: 0.7003
Epoch [1/1], Step [13/37], Loss: 0.7089
Epoch [1/1], Step [14/37], Loss: 0.6868
Epoch [1/1], Step [15/37], Loss: 0.6918
Epoch [1/1], Step [16/37], Loss: 0.6943
Epoch [1/1], Step [17/37], Loss: 0.6896
Epoch [1/1], Step [18/37], Loss: 0.6896
Epoch [1/1], Step [19/37], Loss: 0.6908
Epoch [1/1], Step [20/37], Loss: 0.6928
Epoch [1/1], Step [21/37], Loss: 0.6883
Epoch [1/1], Step [22/37], Loss: 0.6860
Epoch [1/1], Step [23/37], Loss: 0.6902
Epoch [1/1], Step [24/37], Loss: 0.6877
Epoch [1/1], Step [25/37], Loss: 0.6856
Epoch [1/

In [37]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for data_b in dlDataArrtest:
        inputs, labels = data_b[0].to(torch.float32),data_b[1].to(torch.int64)
        outputs = model(inputs)
        # max returns (value ,index)

        _, predicted = torch.max(outputs.data, 1)

        #print(predicted, labels)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]) tensor([1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]) tensor([1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]) tensor([1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]) tensor([1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]) tensor([1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]) tensor([0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]) tensor([0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0,